In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import load_model

from utils.custom_loss import mae_wrap_angle, mse_wrap_angle, angle_diff_deg
from utils.load_data_raw import DataGenerator_raw, load_raw_all_h5
from utils.dataset_split import *
from utils.eval import *
from utils.utils import get_filelist, open_json

NUM_WORKER = 4
custom_obj={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle}

In [ ]:
# load filelist
model_dir = '../data/models_trained/'
filelist = get_filelist(model_dir)

# load data
dset_dir = '/media/feliximmohr/Storage/master_thesis/generated/database/raw/raw_nf10_mid/database_raw_nf10_scaledMM.h5'
feat, targ, ID_ref, pos_t, cond_t, par = load_raw_all_h5(dset_dir)

# load unscaled data
dset_nsc_dir = '/media/feliximmohr/Storage/master_thesis/generated/database/raw/raw_nf10_mid/database_raw_nf10.h5'
feat_nsc, targ_nsc, _, _, _, _ = load_raw_all_h5(dset_nsc_dir)

# create parameter dicts for the test batch generators
params = create_test_params(feat, targ, par, shuffle=False)
params_nsc = create_test_params(feat_nsc, targ_nsc, par, shuffle=False)

cn = feat.columns.tolist()

part = {}
for i in range(len(cond_t)):
    cond = cond_t.iloc[i]
    cond_ids_test, pos_ids_test, subject_ids_test = get_subset_ids([cond[0]], cond_t)
    part[cond[0]] = get_subset_sample_idx(ID_ref, cond_ids_test, pos_ids_test, subject_ids_test)

In [ ]:
# extract model and corresponding history and test partition filenames from filelist
m_flist = [x for x in filelist if ('history' not in x) and ('partition_test' not in x)]
h_flist = ['_' for x in np.zeros(len(m_flist))]
pt_flist = ['_' for x in np.zeros(len(m_flist))]
for file in filelist:
        if ('history' in file):
            name = file.replace('_history.json', '.h5')
            h_flist[m_flist.index(name)] = file
        if ('partition_test' in file):
            name = file.replace('_partition_test.json', '.h5')
            pt_flist[m_flist.index(name)] = file

# seperate model list based on test type (topology/wrapping/normalization)
m_flist_tt = [x for x in m_flist if 'toptest' in x]
m_flist_nt = [x for x in m_flist if 'normtest' in x]
m_flist_wt = [x for x in m_flist if 'wraptest' in x]

h_flist_tt = [h_flist[m_flist.index(i)] for i in m_flist_tt]
h_flist_nt = [h_flist[m_flist.index(i)] for i in m_flist_nt]
h_flist_wt = [h_flist[m_flist.index(i)] for i in m_flist_wt]

pt_flist_tt = [pt_flist[m_flist.index(i)] for i in m_flist_tt]
pt_flist_nt = [pt_flist[m_flist.index(i)] for i in m_flist_nt]
pt_flist_wt = [pt_flist[m_flist.index(i)] for i in m_flist_wt]

In [ ]:
def print_filelist(flist):
    """Print tuple of filelists incl index."""
    if not isinstance(flist,tuple):
        flist = (flist,)
        
    for i in range(len(flist[0])):
        s = ' -' if i<10 else '-'
        for j in range(len(flist)):
            print(i, s, flist[j][i])

# print model list
print_filelist(m_flist)
#print_filelist((m_flist,h_flist,pt_flist))

In [ ]:
def load_m_h_pt(model_dir, m_flist, h_flist, pt_flist):
    """Load models from model list and corresponding history and test partition."""
    m = []
    h = ['_' for x in np.zeros(len(m_flist))]
    p = ['_' for x in np.zeros(len(m_flist))]
    for i,j in enumerate(m_flist):
        m.append(load_model(model_dir + j, custom_objects=custom_obj))
        if h_flist[i] != '_':
            h[i] = open_json(model_dir, h_flist[i])
        if pt_flist[i] != '_':
            p[i] = open_json(model_dir, pt_flist[i])
    return m, h, p

# load models
m_nt, h_nt, pt_nt = load_m_h_pt(model_dir, m_flist_nt, h_flist_nt, pt_flist_nt)
m_wt, h_wt, pt_wt = load_m_h_pt(model_dir, m_flist_wt, h_flist_wt, pt_flist_wt)
m_tt, h_tt, pt_tt = load_m_h_pt(model_dir, m_flist_tt, h_flist_tt, pt_flist_tt)

## Test: normalized vs. not normalized

    Model 1: 128-128-1    m128-128_maew_normtest_neg  vs.  m128-128_maew_normtest_sc


In [ ]:
print_filelist(m_flist_nt)

In [ ]:
# Model 1 not normalized: m128-128_maew_normtest_neg
tbg128_maew_neg = model_complete_eval(m_nt[0], h_nt[0], pt_nt[0], params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m_nt[0], tbg128_maew_neg, b_idx=0)

In [ ]:
# Model 1 normalized: m128-128_maew_normtest_sc
tbg128_maew_sc = model_complete_eval(m_nt[1], h_nt[1], pt_nt[1], params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m_nt[1], tbg128_maew_sc, b_idx=0)

## Test: non-custom vs. custom loss function

    Model 1: 64-32-1      m064-032_mae_wraptest  vs.  m064-032_maew_wraptest
    Model 2: 128-128-1    m128-128_mae_wraptest  vs.  m128-128_maew_wraptest
    Model 3: 256-128-1    m256-128_mae_wraptest  vs.  m256-128_maew_wraptest


In [ ]:
print_filelist(m_flist_wt)

In [ ]:
# Model 1 mae loss: m064-032_mae
tbg64_maew = model_complete_eval(m_wt[0], h_wt[0], p_wt[0], params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m_wt[0], tbg64_maew, b_idx=0)

In [ ]:
# Model 1 custom loss: m64_maew
tbg64_mae = model_complete_eval(m_wt[1], h_wt[1], p_wt[1], params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m_wt[1], tbg64_mae, b_idx=0)

In [ ]:
# Model 2 mae loss: m128-128_mae
tbg128_mae = model_complete_eval(m_wt[2], h_wt[2], p_wt[2], params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m_wt[2], tbg128_mae, b_idx=0)

In [ ]:
# Model 2 custom loss: m128-128_maew
tbg128_maew = model_complete_eval(m_wt[3], h_wt[3], p_wt[3], params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m_wt[3], tbg128_maew, b_idx=0)

In [ ]:
# Model 3 mae loss: m256-128_mae
tbg256_mae = model_complete_eval(m_wt[4], h_wt[4], p_wt[4], params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m_wt[4], tbg256_mae, b_idx=0)

In [ ]:
# Model 3 custom loss: m256-128_maew
tbg256_maew = model_complete_eval(m_wt[5], h_wt[5], p_wt[5], params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m_wt[5], tbg256_maew, b_idx=0)

## Test: Model/Net Topology

    TODO
    Model 1:         128-128-1
    Model 1 + BN :   128-BN-128-BN-1
    Model 2:         128-128-64-1
    Model 3:         512-256-1
    Model 4:         32-16-1

In [ ]:
print_filelist(m_flist_tt)

In [ ]:
# Model 1: m128_tt
tbg128_tt = model_complete_eval(m128_tt, h128_tt, pt128_tt, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_tt, tbg128_tt, b_idx=0)

In [ ]:
# Model 1 + BN: m128bn_bs2048_tt
tbg128bn_bs2048_tt = model_complete_eval(m128bn_bs2048_tt, h128bn_bs2048_tt, pt128bn_bs2048_tt, params, batch_size=2048, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128bn_bs2048_tt, tbg128bn_bs2048_tt, b_idx=0)

In [ ]:
# Model 2: m128_128_64_tt
tbg128_128_64_tt = model_complete_eval(m128_128_64_tt, h128_128_64_tt, pt128_128_64_tt, params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_128_64_tt, tbg128_128_64_tt, b_idx=0)

In [ ]:
# Model 3: m512_256_tt
tbg512_256_tt = model_complete_eval(m512_256_tt, h512_256_tt, pt512_256_tt, params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m512_256_tt, tbg512_256_tt, b_idx=0)

In [ ]:
# Model 4: m32_16_tt not normalized?
tbg32_16_tt = model_complete_eval(m32_16_tt, h32_16_tt, pt32_16_tt, params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
pred, y = model_pred_on_gen_batch(m32_16_tt, tbg32_16_tt, b_idx=0)

### Results

In [ ]:
# Model 1: m128_tt
mae_p, mse_p, loc_p = model_eval_pos(m128_tt, h128_tt, part['NFCHOA_R006'], params, ID_ref, batch_size=1000, workers=NUM_WORKER)

## Error Comparison

In [ ]:
# get IDs of subset of data per position of one condition
part_x_M027, pos_ids = get_pos_IDs(part['NFCHOA_M027'],ID_ref)
part_x_R006, _ = get_pos_IDs(part['NFCHOA_R006'],ID_ref)

params_t = params.copy()
params_t['batch_size'] = 1000

In [ ]:
# calculate MSE = squared bias + variance
n_model = len(m_flist_tt)
MSE_R006 = np.zeros(n_model)
tau_sq_R006 = np.zeros(n_model)
sigma_sq_R006 = np.zeros(n_model)
MSE_M027 = np.zeros(n_model)
tau_sq_M027 = np.zeros(n_model)
sigma_sq_M027 = np.zeros(n_model)

for i in np.arange(n_model):
    model = load_model(model_dir + m_flist_tt[i], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
    MSE_R006[i], tau_sq_R006[i], sigma_sq_R006[i], _, _ = calc_errors_m(model, part_x_R006, params_t, pos_ids, verbose=0, workers=NUM_WORKER)
    MSE_M027[i], tau_sq_M027[i], sigma_sq_M027[i], _, _ = calc_errors_m(model, part_x_M027, params_t, pos_ids, verbose=0, workers=NUM_WORKER)
    print(i)

In [ ]:
m_nlist = []
loss_f = []
model_str = []
test_dset = []
bs = []
for i in np.arange(n_model):
    strn = m_flist[i].split('_')
    loss_f.append('{:<5}'.format(strn[1]))
    model_str.append('{:<15}'.format(strn[0]))
    test_dset.append('{:<15}'.format(strn[4]))
    bs.append('{:<5}'.format(strn[3]))
    m_nlist.append(strn[0]+'_'+strn[1]+'_'+strn[4])

data = {
    'model':model_str,
    'loss_f':loss_f,
    'test_dset':test_dset,
    'bs':bs,
    'MSE_R006' : MSE_R006,
    'sq_bias_R006': tau_sq_R006,
    'variance_R006': sigma_sq_R006,
    'MSE_M027': MSE_M027,
    'sq_bias_M027': tau_sq_M027,
    'variance_M027': sigma_sq_M027    
}
df = pd.DataFrame(data)